# Outlook

In this notebook, using BBRL, we code the DDPG algorithm. To understand this code, you need [to know more about BBRL](https://colab.research.google.com/drive/1_yp-JKkxh_P8Yhctulqm0IrLbE41oK1p?usp=sharing). You should first have a look at [the BBRL interaction model](https://colab.research.google.com/drive/1gSdkOBPkIQi_my9TtwJ-qWZQS0b2X7jt?usp=sharing), then [a first example](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing) and, most importantly, [details about the AutoResetGymAgent](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing).

The DDPG algorithm is explained in [this video](https://www.youtube.com/watch?v=0D6a0a1HTtc) and you can also read [the corresponding slides](http://pages.isir.upmc.fr/~sigaud/teach/ddpg.pdf).

## Installation and Imports

### Installation

The BBRL library is [here](https://github.com/osigaud/bbrl).

This is OmegaConf that makes it possible that by just defining the `def run_dqn(cfg):` function and then executing a long `params = {...}` variable at the bottom of this colab, the code is run with the parameters without calling an explicit main.

More precisely, the code is run by calling

`config=OmegaConf.create(params)`

`run_dqn(config)`

at the very bottom of the colab, after starting tensorboard.

try:
    from easypip import easyimport
except:
    # !pip install easypip
    from easypip import easyimport

import os
import functools
import time
from typing import Tuple

OmegaConf = easyimport("omegaconf").OmegaConf
torch = easyimport("torch")
bbrl_gym = easyimport("bbrl_gym") 
bbrl = easyimport("bbrl")
```

<!-- #region id="m4kV9pWV3wRe" -->
### Imports
<!-- #endregion -->

<!-- #region id="caqhJYbe5YcO" -->
Below, we import standard python packages, pytorch packages and gym environments.
<!-- #endregion -->

<!-- #region id="4l7sTVXbJBE_" -->
[OpenAI gym](https://gym.openai.com/) is a collection of benchmark environments to evaluate RL algorithms.
<!-- #endregion -->

```{python id="vktQB-AO5biu"}
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

import gym
```

### BBRL imports

In [ ]:
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class

# The workspace is the main class in BBRL, this is where all data is collected and stored
from bbrl.workspace import Workspace

# Agents(agent1,agent2,agent3,...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent over multiple timesteps in the workspace, 
# or until a given condition is reached
from bbrl.agents import Agents, RemoteAgent, TemporalAgent

# AutoResetGymAgent is an agent able to execute a batch of gym environments
# with auto-resetting. These agents produce multiple variables in the workspace: 
# ’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/done’, ’env/initial_state’, ’env/cumulated_reward’, 
# ... When called at timestep t=0, then the environments are automatically reset. 
# At timestep t>0, these agents will read the ’action’ variable in the workspace at time t − 1
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
# Not present in the A2C version...
from bbrl.utils.logger import TFLogger
from bbrl.utils.replay_buffer import ReplayBuffer

## Definition of agents

The [DDPG](https://arxiv.org/pdf/1509.02971.pdf) algorithm is an actor critic algorithm. We use the standard function for building the neural networks that will play the role of the actor and the critic.

In [ ]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

To implement the gym environment, we use the same approach as usual.

In [ ]:
def make_gym_env(env_name):
    return gym.make(env_name)

The critic is a neural network taking the state $s$ and action $a$ as input, and its output layer has a unique neuron whose value is the value of being in that state and performing that action $Q(s,a)$.

As usual, the ```forward(...)``` function is used to write Q-values in the workspace from time indexes, whereas the ```predict_value(...)``` function` is used in other contexts, such as plotting a view of the Q function.

In [ ]:
class ContinuousQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.is_q_function = True
        self.model = build_mlp(
            [state_dim + action_dim] + list(hidden_layers) + [1], activation=nn.ReLU()
        )

    def forward(self, t, detach_actions=False):
        obs = self.get(("env/env_obs", t))
        action = self.get(("action", t))
        if detach_actions:
            action = action.detach()
        osb_act = torch.cat((obs, action), dim=1)
        q_value = self.model(osb_act)
        self.set(("q_value", t), q_value)

    def predict_value(self, obs, action):
        osb_act = torch.cat((obs, action), dim=0)
        q_value = self.model(osb_act)
        return q_value

The actor is also a neural network, it takes a state $s$ as input and outputs an action $a$.

In [ ]:
class ContinuousDeterministicActor(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        layers = [state_dim] + list(hidden_layers) + [action_dim]
        self.model = build_mlp(
            layers, activation=nn.ReLU(), output_activation=nn.Tanh()
        )

    def forward(self, t):
        obs = self.get(("env/env_obs", t))
        action = self.model(obs)
        self.set(("action", t), action)

    def predict_action(self, obs, stochastic):
        assert (
            not stochastic
        ), "ContinuousDeterministicActor cannot provide stochastic predictions"
        return self.model(obs)

### Creating an Exploration method

In the continuous action domain, basic exploration differs from the methods used in the discrete action domain. Here we generally add some Gaussian noise to the output of the actor.

In [ ]:
from torch.distributions import Normal

In [ ]:
class AddGaussianNoise(Agent):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, t, **kwargs):
        act = self.get(("action", t))
        dist = Normal(act, self.sigma)
        action = dist.sample()
        self.set(("action", t), action)



In [the original DDPG paper](https://arxiv.org/pdf/1509.02971.pdf), the authors rather used the more sophisticated Ornstein-Uhlenbeck noise where noise is correlated between one step and the next.

In [ ]:
class AddOUNoise(Agent):
    """
    Ornstein Uhlenbeck process noise for actions as suggested by DDPG paper
    """

    def __init__(self, std_dev, theta=0.15, dt=1e-2):
        self.theta = theta
        self.std_dev = std_dev
        self.dt = dt
        self.x_prev = 0

    def forward(self, t, **kwargs):
        act = self.get(("action", t))
        x = (
            self.x_prev
            + self.theta * (act - self.x_prev) * self.dt
            + self.std_dev * math.sqrt(self.dt) * torch.randn(act.shape)
        )
        self.x_prev = x
        self.set(("action", t), x)



### Training and evaluation environments

We build two environments: one for training and another one for evaluation.

For training, it is more efficient to use an AutoResetGymAgent, as we do not want to waste time if the task is done in an environment sooner than in the others.

By contrast, for evaluation, we just need to perform a fixed number of episodes (for statistics), thus it is more convenient to use a NoAutoResetGymAgent with a set of environments and just run one episode in each environment. Thus we can use the `env/done` stop variable and take the average over the cumulated reward of all environments.


See [this notebook](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing) for explanations about agents and environment agents.

In [ ]:
def get_env_agents(cfg):
    train_env_agent = AutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )
    eval_env_agent = NoAutoResetGymAgent(
    get_class(cfg.gym_env),
    get_arguments(cfg.gym_env),
    cfg.algorithm.nb_evals,
    cfg.algorithm.seed,
    )
    return train_env_agent, eval_env_agent

### Create the DDPG agent

In this function we create the critic and the actor, but also an exploration agent to add noise and a target critic. The version below does not use a target actor as it proved hard to tune, but such a target actor is used in the original paper.

In [ ]:
# Create the DDPG Agent
def create_ddpg_agent(cfg, train_env_agent, eval_env_agent):
    obs_size, act_size = train_env_agent.get_obs_and_actions_sizes()
    critic = ContinuousQAgent(
        obs_size, cfg.algorithm.architecture.critic_hidden_size, act_size
    )
    target_critic = copy.deepcopy(critic)
    actor = ContinuousDeterministicActor(
        obs_size, cfg.algorithm.architecture.actor_hidden_size, act_size
    )
    # target_actor = copy.deepcopy(actor) # not used in practice, though described in the paper
    noise_agent = AddGaussianNoise(cfg.algorithm.action_noise) # alternative : AddOUNoise
    tr_agent = Agents(train_env_agent, actor, noise_agent)  
    ev_agent = Agents(eval_env_agent, actor)

    # Get an agent that is executed on a complete workspace
    train_agent = TemporalAgent(tr_agent)
    eval_agent = TemporalAgent(ev_agent)
    train_agent.seed(cfg.algorithm.seed)
    return train_agent, eval_agent, actor, critic, target_critic  # , target_actor

### The Logger class

Explanations for the logger were already given in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV?usp=sharing).

In [ ]:
class Logger():

  def __init__(self, cfg):
    self.logger = instantiate_class(cfg.logger)

  def add_log(self, log_string, loss, epoch):
    self.logger.add_scalar(log_string, loss.item(), epoch)

  # Log losses
  def log_losses(self, cfg, epoch, critic_loss, entropy_loss, a2c_loss):
    self.add_log("critic_loss", critic_loss, epoch)
    self.add_log("entropy_loss", entropy_loss, epoch)
    self.add_log("a2c_loss", a2c_loss, epoch)



### Setup the optimizers

We use two separate optimizers to tune the parameters of the actor and the critic separately. That makes it possible to use a different learning rate for the actor and the critic.

In [ ]:
# Configure the optimizers
def setup_optimizers(cfg, actor, critic):
    actor_optimizer_args = get_arguments(cfg.actor_optimizer)
    parameters = actor.parameters()
    actor_optimizer = get_class(cfg.actor_optimizer)(parameters, **actor_optimizer_args)
    critic_optimizer_args = get_arguments(cfg.critic_optimizer)
    parameters = critic.parameters()
    critic_optimizer = get_class(cfg.critic_optimizer)(
        parameters, **critic_optimizer_args
    )
    return actor_optimizer, critic_optimizer

### Compute critic loss

Detailed explanations of the function to compute the critic loss when using a NoAutoResetGymAgent are given in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV?usp=sharing).

The case where we use the AutoResetGymAgent is very similar, but we need to specify that we use the first part of the Q-values (`q_values[0]`) for representing $Q(s_t,a_t)$ and the second part (`q_values[1]`) for representing $Q(s_{t+1},a)$, as these values are stored into a transition model. Then the values used to compute the target in the critic update are stored into ```target_q_values```.

In [ ]:
def compute_critic_loss(cfg, reward, must_bootstrap, q_values, target_q_values):
    # Compute temporal difference
    q_next = target_q_values
    target = (
        reward[:-1].squeeze()
        + cfg.algorithm.discount_factor * q_next.squeeze(-1) * must_bootstrap.int()
    )
    # Compute critic loss
    mse = nn.MSELoss()
    critic_loss = mse(target, q_values.squeeze(-1))
    return critic_loss

To update the target critic, one uses the following equation:

$\theta' \leftarrow \tau \theta + (1- \tau) \theta'$


where $\theta$ is the vector of parameters of the critic, and $\theta'$ is the vector of parameters of the target critic.

The `soft_update_params(...)` function is in charge of performing this soft update.

In [ ]:
def soft_update_params(net, target_net, tau):
    for param, target_param in zip(net.parameters(), target_net.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

### Compute actor loss

The actor loss is straightforward. We want the actor to maximize Q-values, thus we minimize the mean of negated Q-values.

In [ ]:
def compute_actor_loss(q_values):
    return -q_values.mean()

## Main training loop

### Agent execution

This is the tricky part with BBRL, the one we need to understand in detail. The difficulty lies in the copy of the last step and the way to deal with the n_steps return.

The call to `train_agent(workspace, t=1, n_steps=cfg.algorithm.n_timesteps - 1, stochastic=True)` makes the agent run a number of steps in the workspace. In practice, it calls the [`__call__(...)`](https://github.com/osigaud/bbrl/blob/master/bbrl/agents/agent.py#L54) function which makes a forward pass of the agent network using the workspace data and updates the workspace accordingly.

Now, if we start at the first epoch (`epoch=0`), we start from the first step (`t=0`). But when subsequently we perform the next epochs (`epoch>0`), we must not forget to cover the transition at the border between the previous epoch and the current epoch. To avoid this risk, we copy the information from the last time step of the previous epoch into the first time step of the next epoch. This is explained in more details in [a previous notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5).

A [previous notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5) explains a lot of these details. In particular, read it to understand the `execute_agents(...)` function, the `transition_workspace = train_workspace.get_transitions()` line and the computation of `must_bootstrap`.

Note that we `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()` lines. 

`optimizer.zero_grad()` is necessary to cancel all the gradients computed at the previous iterations

Note the way we count the steps, to properly ignore the steps corresponding to a transition from an episode to the next.

Note also that every ```cfg.algorithm.eval_interval```, we evaluate the current agent and save it and plot it if its performance is the new best performance.

In [ ]:
from bbrl.visu.visu_policies import plot_policy
from bbrl.visu.visu_critics import plot_critic

In [ ]:
def run_ddpg(cfg):
    # 1)  Build the  logger
    logger = Logger(cfg)
    best_reward = -10e9

    # 2) Create the environment agent
    train_env_agent = AutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )
    eval_env_agent = NoAutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.nb_evals,
        cfg.algorithm.seed,
    )

    # 3) Create the DDPG Agent
    (
        train_agent,
        eval_agent,
        actor,
        critic,
        # target_actor,
        target_critic,
    ) = create_ddpg_agent(cfg, train_env_agent, eval_env_agent)
    ag_actor = TemporalAgent(actor)
    # ag_target_actor = TemporalAgent(target_actor)
    q_agent = TemporalAgent(critic)
    target_q_agent = TemporalAgent(target_critic)

    train_workspace = Workspace()
    rb = ReplayBuffer(max_size=cfg.algorithm.buffer_size)

    # Configure the optimizer
    actor_optimizer, critic_optimizer = setup_optimizers(cfg, actor, critic)
    nb_steps = 0
    tmp_steps = 0

    # Training loop
    for epoch in range(cfg.algorithm.max_epochs):
        # Execute the agent in the workspace
        if epoch > 0:
            train_workspace.zero_grad()
            train_workspace.copy_n_last_steps(1)
            train_agent(train_workspace, t=1, n_steps=cfg.algorithm.n_steps - 1)
        else:
            train_agent(train_workspace, t=0, n_steps=cfg.algorithm.n_steps)

        transition_workspace = train_workspace.get_transitions()
        action = transition_workspace["action"]
        nb_steps += action[0].shape[0]
        rb.put(transition_workspace)
        rb_workspace = rb.get_shuffled(cfg.algorithm.batch_size)

        done, truncated, reward, action = rb_workspace[
            "env/done", "env/truncated", "env/reward", "action"
        ]
        # print(f"done {done}, reward {reward}, action {action}")
        if nb_steps > cfg.algorithm.learning_starts:
            # Determines whether values of the critic should be propagated
            # True if the episode reached a time limit or if the task was not done
            # See https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing
            must_bootstrap = torch.logical_or(~done[1], truncated[1])

            # Critic update
            # compute q_values: at t, we have Q(s,a) from the (s,a) in the RB
            q_agent(rb_workspace, t=0, n_steps=1)
            q_values = rb_workspace["q_value"]
            # print(f"q_values ante : {q_values}")

            with torch.no_grad():
                # replace the action at t+1 in the RB with \pi(s_{t+1}), to compute Q(s_{t+1}, \pi(s_{t+1}) below
                ag_actor(rb_workspace, t=1, n_steps=1)
                # compute q_values: at t+1 we have Q(s_{t+1}, \pi(s_{t+1})
                target_q_agent(rb_workspace, t=1, n_steps=1)
                # q_agent(rb_workspace, t=1, n_steps=1)
            # finally q_values contains the above collection at t=0 and t=1
            post_q_values = rb_workspace["q_value"]
            # print(f"q_values post : {post_q_values[1]}")

            # Compute critic loss
            critic_loss = compute_critic_loss(
                cfg, reward, must_bootstrap, q_values[0], post_q_values[1]
            )
            logger.add_log("critic_loss", critic_loss, nb_steps)
            critic_optimizer.zero_grad()
            critic_loss.backward()
            torch.nn.utils.clip_grad_norm_(
                critic.parameters(), cfg.algorithm.max_grad_norm
            )
            critic_optimizer.step()

            # Actor update
            # Now we determine the actions the current policy would take in the states from the RB
            ag_actor(rb_workspace, t=0, n_steps=1)
            # We determine the Q values resulting from actions of the current policy
            q_agent(rb_workspace, t=0, n_steps=1)
            # and we back-propagate the corresponding loss to maximize the Q values
            q_values = rb_workspace["q_value"]
            actor_loss = compute_actor_loss(q_values)
            logger.add_log("actor_loss", actor_loss, nb_steps)
            # if -25 < actor_loss < 0 and nb_steps > 2e5:
            actor_optimizer.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(
                actor.parameters(), cfg.algorithm.max_grad_norm
            )
            actor_optimizer.step()
            # Soft update of target q function
            tau = cfg.algorithm.tau_target
            soft_update_params(critic, target_critic, tau)
            # soft_update_params(actor, target_actor, tau)

        if nb_steps - tmp_steps > cfg.algorithm.eval_interval:
            tmp_steps = nb_steps
            eval_workspace = Workspace()  # Used for evaluation
            eval_agent(eval_workspace, t=0, stop_variable="env/done")
            rewards = eval_workspace["env/cumulated_reward"][-1]
            mean = rewards.mean()
            logger.add_log("reward", mean, nb_steps)
            print(f"nb_steps: {nb_steps}, reward: {mean}")

            if mean > best_reward:
                best_reward = mean

                if cfg.save_best:
                    directory = "./ddpg_agent/"
                    if not os.path.exists(directory):
                        os.makedirs(directory)
                    filename = directory + "ddpg_" + str(mean.item()) + ".agt"
                    eval_agent.save_model(filename)

                if cfg.plot_agents:
                    plot_policy(
                        actor,
                        eval_env_agent,
                        "./ddpg_plots/",
                        cfg.gym_env.env_name,
                        best_reward,
                        stochastic=False,
                    )
                    plot_critic(
                        q_agent.agent,
                        eval_env_agent,
                        "./ddpg_plots/",
                        cfg.gym_env.env_name,
                        best_reward,
                    )



## Definition of the parameters

The logger is defined as `bbrl.utils.logger.TFLogger` so as to use a tensorboard visualisation.

In [ ]:
params={
  "save_best": False,
  # Set to true to have an insight on the learned policy
  # (but slows down the evaluation a lot!)
  "plot_agents": True,
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./tblogs/ddpg/ddpg-" + str(time.time()),
    "cache_size": 10000,
    "every_n_seconds": 10,
    "verbose": False,    
    },

  "algorithm":{
    "seed": 1,
    "max_grad_norm": 0.5,
    "epsilon": 0.02,
    "n_envs": 1,
    "n_steps": 100,
    "eval_interval": 2000,
    "nb_evals": 10,
    "gae": 0.8,
    "max_epochs": 21000,
    "discount_factor": 0.98,
    "buffer_size": 2e5,
    "batch_size": 64,
    "tau_target": 0.05,
    "learning_starts": 10000,
    "action_noise": 0.1,
    "architecture":{
        "actor_hidden_size": [400, 300],
        "critic_hidden_size": [400, 300],
        },
  },
  "gym_env":{
    "classname": "__main__.make_gym_env",
    "env_name": "Pendulum-v1",
  },
  "actor_optimizer":{
    "classname": "torch.optim.Adam",
    "lr": 1e-3,
  },
  "critic_optimizer":{
    "classname": "torch.optim.Adam",
    "lr": 1e-3,
  }
}

### Launching tensorboard to visualize the results

In [ ]:
# For Colab - otherwise, it is easier and better to launch tensorboard from
# the terminal
if get_ipython().__class__.__module__ == "google.colab._shell":
    %load_ext tensorboard
    %tensorboard --logdir ./tmp
else:
    import sys
    import os
    import os.path as osp
    print(f"Launch tensorboard from the shell:\n{osp.dirname(sys.executable)}/tensorboard --logdir={os.getcwd()}/tmp")

In [ ]:
config=OmegaConf.create(params)
torch.manual_seed(config.algorithm.seed)
run_ddpg(config)

## What's next?

Starting from the above version , you sould code [the TD3 algorithm](http://proceedings.mlr.press/v80/fujimoto18a/fujimoto18a.pdf).

For that, you need to use two critics (and two target critics) and always take the minimum output between the two when you ask for the Q-value of a (state, action) pair.

In more detail, you have to do the following:
- replace the single critic and corresponding target critic with two critics and target critics (name them ```critic_1, critic_2, target_critic_1, target_critic_2```)
- get the q-values and target q-values corresponding to all these critics.
- then the target q-values you should consider to update the critic should be the min over the target q-values at each step (use ```torch.min(...)``` to get this min over a sequence of data).
- to update the actor, do it with the q-values of an arbitrarily chosen critic, e.g. critic_1.


In [ ]:
# À compléter...  
assert False, 'Code non implémenté'


## Experimental comparison

Take an environment where the over-estimation bias may matter, and compare the performance of DDPG and TD3. Visualize the Q-value long before convergence to see whether indeed DDPG overestimates the Q-values with respect to TD3.